In [1]:
from collections import deque

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

np.set_printoptions(threshold=np.inf)

In [2]:
directions = {(-1, 0): 'N', (1, 0): 'S', (0, -1): 'W', (0, 1): 'E'}

In [3]:
def parse_input(file):
    with open(file) as file_in:
        grid = file_in.read().splitlines()

    grid = np.array([list(row) for row in grid])

    S_coords = tuple(np.argwhere(grid == 'S')[0].tolist())
    E_coords = tuple(np.argwhere(grid == 'E')[0].tolist())
    dot_coords = set([(x.item(), y.item()) for x, y in np.argwhere(grid == '.')])
    wall_coords = set([(x.item(), y.item()) for x, y in np.argwhere(grid == '#')])

    return S_coords, E_coords, wall_coords

In [9]:
def get_min_score(S_coords, E_coords, wall_coords):
    start = (S_coords, 'E', 0)
    queue = deque([start])
    scores = []
    best_state = {}

    while queue:
        (x_current, y_current), dir_current, score = queue.popleft()
        for dx, dy in directions:
            x_next, y_next = x_current + dx, y_current + dy 

            if (x_next, y_next) == E_coords:
                # If reaches E
                scores.append(score + 1)
                continue

            if (x_next, y_next) not in wall_coords:
                # If not visited yet
                dir_next = directions[(dx, dy)]
                if dir_next == dir_current:
                    new_score = score + 1
                else:
                    new_score = score + 1001

                # Keep state to enable tiles to be revisited if the path's score is better
                state = ((x_next, y_next), dir_next)
                if state not in best_state or new_score < best_state[state]:
                    best_state[state] = new_score
                    queue.append(((x_next, y_next), dir_next, new_score))

    return min(scores)

In [10]:
def main1(file):
    S_coords, E_coords, wall_coords = parse_input(file)
    min_score = get_min_score(S_coords, E_coords, wall_coords)
    return min_score

In [11]:
assert main1('example1.txt') == 7036
assert main1('example2.txt') == 11048

In [12]:
main1('input.txt')

102460